In [1]:
import findspark
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')
import pyspark
sc=pyspark.SparkContext()
from pyspark.sql import *
sqlContext = SQLContext(sc)
from pyspark.sql.functions import udf,col,concat_ws
spark=SparkSession.builder.appName("Pyspark Demo").config("sparl.some.config.option","some-value").getOrCreate()
import re
from pyspark.sql import Row
from pyspark.sql.functions import monotonically_increasing_id 
from pyspark.sql.functions import udf, col, lower, regexp_replace
from pyspark.ml.feature import Tokenizer, StopWordsRemover, RegexTokenizer
from pyspark.sql.functions import trim
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.evaluation import MultilabelMetrics
from pyspark.ml.feature import CountVectorizer
import pandas as pd
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.types import DoubleType
from functools import reduce

In [2]:
pd_train_data= pd.read_csv("train.csv")
train_data=spark.createDataFrame(pd_train_data,schema=None,verifySchema=True)
train_data.printSchema()


root
 |-- movie_id: long (nullable = true)
 |-- movie_name: string (nullable = true)
 |-- plot: string (nullable = true)
 |-- genre: string (nullable = true)



In [3]:
pd_test_data= pd.read_csv("test.csv")
test_data=spark.createDataFrame(pd_test_data,schema=None,verifySchema=True)
test_data.printSchema()


root
 |-- movie_id: long (nullable = true)
 |-- movie_name: string (nullable = true)
 |-- plot: string (nullable = true)



In [4]:
pd_mapping = pd.read_csv("mapping.csv")
mapping=spark.createDataFrame(pd_mapping)
mapping.printSchema()

root
 |-- Unnamed: 0: long (nullable = true)
 |-- 0: string (nullable = true)



In [5]:
maplist = [row for row in mapping.select('0').collect()]

In [6]:
def map_list(x):
    z=x.collect()
    map_data=[]
    for j in range(0,x.count()):
        res=[]
        genre_list=z[j][0].replace('[','').replace(']','').replace("'","").split(',')
        genre=[]
        for g in genre_list:
            genre.append(g.strip())
        for k in enumerate(maplist):
            if k[1][0] in genre:
                res.append(1)
            else:
                res.append(0)
#         listToStr = ' '.join([str(elem) for elem in res]) 
        map_data.append(res)
    rdd1 = sc.parallelize(map_data)
    row_rdd = rdd1.map(lambda x: Row(x))
    return sqlContext.createDataFrame(row_rdd,['map_list'])
map_data = map_list(train_data.select('genre'))
map_data.head()

Row(map_list=[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [7]:
train_data_new = train_data.withColumn("index", monotonically_increasing_id())
map_data_new = map_data.withColumn("index", monotonically_increasing_id())

train_data = train_data_new.join(map_data_new, on=['index'], how='inner')

train_data.show()

+-----+--------+--------------------+--------------------+--------------------+--------------------+
|index|movie_id|          movie_name|                plot|               genre|            map_list|
+-----+--------+--------------------+--------------------+--------------------+--------------------+
|   26|18549958|          Love, Mary|Mary was a lonely...|           ['Drama']|[1, 0, 0, 0, 0, 0...|
|   29| 3150865| Yankee Doodle Daffy|Porky Pig is tryi...|['Animation', 'Co...|[0, 1, 0, 0, 0, 0...|
|  474| 4345210|     Learning to Lie|Beginning in West...|    ['Romance Film']|[0, 0, 1, 0, 0, 0...|
|  964|14072881|   Boys' Reformatory|Seventeen-year-ol...|['Black-and-white...|[1, 0, 0, 0, 0, 0...|
| 1677| 4711636|The Last Flight o...|A jaded pilot nam...|       ['Adventure']|[0, 0, 0, 0, 0, 0...|
| 1697|27181651|              Esther|Hadassah , a beau...|['Drama', 'Crime ...|[1, 0, 1, 0, 0, 0...|
| 1806|26551541|      Triple Trouble|The boys are on t...|['Crime Fiction',...|[0, 1, 0, 0,

In [8]:
def process_plot(data,mode):
    # Convert text to lower case
    data = data.withColumn('plot', (lower(col('plot')).alias('plot')))

    # Tokenize text
    tokenizer = RegexTokenizer(inputCol='plot', outputCol='plot_token',pattern="\\W")
    data = tokenizer.transform(data)


    # Remove stop words
    stopwords = StopWordsRemover(inputCol='plot_token', outputCol='plot_stop')
    data = stopwords.transform(data)

    # Remove blank strings from plot tokens
    data=data.withColumn("plot_stop", concat_ws(",", "plot_stop"))
    data = data.withColumn('plot_stop', regexp_replace('plot_stop', ',', ' '))
    data = data.withColumn("plot_stop", trim(regexp_replace('plot_stop', " +", " ")))
    tokenizer = RegexTokenizer(inputCol='plot_stop', outputCol='plot_output',pattern="\\W")
    data = tokenizer.transform(data)
    if (mode=='train'):
        data = data.select('movie_id','plot_output','map_list')
    if (mode=='test'):
        data = data.select('movie_id','plot_output')
    return data


In [9]:
train_data=process_plot(train_data,'train')
train_data.show()

+--------+--------------------+--------------------+
|movie_id|         plot_output|            map_list|
+--------+--------------------+--------------------+
|18549958|[mary, lonely, po...|[1, 0, 0, 0, 0, 0...|
| 3150865|[porky, pig, tryi...|[0, 1, 0, 0, 0, 0...|
| 4345210|[beginning, west,...|[0, 0, 1, 0, 0, 0...|
|14072881|[seventeen, year,...|[1, 0, 0, 0, 0, 0...|
| 4711636|[jaded, pilot, na...|[0, 0, 0, 0, 0, 0...|
|27181651|[hadassah, beauti...|[1, 0, 1, 0, 0, 0...|
|26551541|[boys, way, home,...|[0, 1, 0, 0, 1, 0...|
|24070825|[film, takes, pla...|[1, 0, 1, 0, 0, 1...|
|30897463|[film, tells, sto...|[0, 0, 1, 0, 0, 0...|
| 5349196|[arul, vikram, wo...|[1, 0, 0, 0, 0, 1...|
|32751781|[australian, coun...|[1, 0, 0, 0, 0, 0...|
|28733517|[post, ww2, londo...|[1, 0, 1, 0, 0, 0...|
|11453926|[charlie, wife, w...|[0, 1, 0, 0, 0, 0...|
|16190026|[july, 2012, gold...|[0, 0, 0, 0, 0, 0...|
|11517297|[betty, dallas, p...|[0, 0, 0, 0, 0, 0...|
| 3492883|[emperor, kuzco, ...|[0, 0, 0, 0, 0,

In [10]:
test_data=process_plot(test_data,'test')
test_data.show()

+--------+--------------------+
|movie_id|         plot_output|
+--------+--------------------+
| 1335380|[film, based, eve...|
|29062594|[group, teenagers...|
| 9252321|[story, zulu, fam...|
|13455076|[stooges, play, t...|
|24165951|[soldier, fortune...|
| 1925869|[set, northwester...|
|10799612|[like, many, moni...|
|28238240|[mickey, scorpion...|
|17124781|[desert, wilderne...|
|28207941|[bimbo, koko, sig...|
|19174305|[tahaan, lives, g...|
|18392317|[betty, startled,...|
|34420857|[nirmal, karthik,...|
| 4039635|[group, journalis...|
| 8034072|[vaibhavari, saha...|
| 4016437|[1947, movie, nar...|
| 1520023|[ninja, resurrect...|
|24589422|[spring, 1946, iv...|
|35068740|[muthu, prabhu, v...|
|21132951|[vishwanathan, in...|
+--------+--------------------+
only showing top 20 rows



## Term Document Matrix

In [25]:
cv = CountVectorizer(inputCol="plot_output", outputCol="features",vocabSize=500, minDF=20)
basic_model = cv.fit(train_data)
train_data_basic = basic_model.transform(train_data)
train_data_basic.show()

+--------+--------------------+--------------------+--------------------+
|movie_id|         plot_output|            map_list|            features|
+--------+--------------------+--------------------+--------------------+
|18549958|[mary, lonely, po...|[1, 0, 0, 0, 0, 0...|(500,[1,14,21,48,...|
| 3150865|[porky, pig, tryi...|[0, 1, 0, 0, 0, 0...|(500,[1,11,15,17,...|
| 4345210|[beginning, west,...|[0, 0, 1, 0, 0, 0...|(500,[0,3,6,7,9,1...|
|14072881|[seventeen, year,...|[1, 0, 0, 0, 0, 0...|(500,[0,5,8,11,12...|
| 4711636|[jaded, pilot, na...|[0, 0, 0, 0, 0, 0...|(500,[1,2,4,8,10,...|
|27181651|[hadassah, beauti...|[1, 0, 1, 0, 0, 0...|(500,[0,1,2,5,6,7...|
|26551541|[boys, way, home,...|[0, 1, 0, 0, 1, 0...|(500,[6,7,12,15,2...|
|24070825|[film, takes, pla...|[1, 0, 1, 0, 0, 1...|(500,[0,2,4,5,7,8...|
|30897463|[film, tells, sto...|[0, 0, 1, 0, 0, 0...|(500,[4,6,8,10,11...|
| 5349196|[arul, vikram, wo...|[1, 0, 0, 0, 0, 1...|(500,[0,3,10,13,1...|
|32751781|[australian, coun...|[1, 0, 

In [26]:
cv = CountVectorizer(inputCol="plot_output", outputCol="features",vocabSize=500, minDF=20)
basic_model_test = cv.fit(test_data)
test_data_basic = basic_model_test.transform(test_data)
test_data_basic.show()

+--------+--------------------+--------------------+
|movie_id|         plot_output|            features|
+--------+--------------------+--------------------+
| 1335380|[film, based, eve...|(500,[0,3,6,7,8,9...|
|29062594|[group, teenagers...|(500,[7,12,56,67,...|
| 9252321|[story, zulu, fam...|(500,[4,7,8,10,13...|
|13455076|[stooges, play, t...|(500,[0,76,78,79,...|
|24165951|[soldier, fortune...|(500,[19,392,439]...|
| 1925869|[set, northwester...|(500,[0,1,2,3,4,5...|
|10799612|[like, many, moni...|(500,[0,5,6,9,12,...|
|28238240|[mickey, scorpion...|(500,[13,21,191,2...|
|17124781|[desert, wilderne...|(500,[4,8,11,13,1...|
|28207941|[bimbo, koko, sig...|(500,[55,56,222,3...|
|19174305|[tahaan, lives, g...|(500,[1,3,4,7,8,1...|
|18392317|[betty, startled,...|(500,[6,7,16,20,2...|
|34420857|[nirmal, karthik,...|(500,[3,5,8,9,17,...|
| 4039635|[group, journalis...|(500,[0,1,6,16,26...|
| 8034072|[vaibhavari, saha...|(500,[2,3,9,10,12...|
| 4016437|[1947, movie, nar...|(500,[0,2,3,5,9

## Basic Model

In [27]:
dlist = train_data_basic.columns
train_data_basic=train_data_basic.select(dlist+[(col("map_list")[x]).alias("genre"+str(x+1)) for x in range(0, 20)])


In [28]:
# (trainset_basic, valset_basic) = train_data_basic.randomSplit([0.8, 0.2], seed = 121)

In [29]:
pred_list=[]
for i in range(1,21):
    
    lr = LogisticRegression(featuresCol = 'features', labelCol='genre%s'%i,maxIter=5)
    lrModel = lr.fit(train_data_basic)
    predictions_test = lrModel.transform(test_data_basic)
    predictions_test = predictions_test.withColumn("prediction", predictions_test["prediction"].cast(IntegerType()))
    predictions_test = (predictions_test.withColumnRenamed('prediction','prediction%s'%i))
    pred_list.append(predictions_test.select('prediction%s'%i).withColumn("index", monotonically_increasing_id()))
    print(i)
movie_list=predictions_test.select('movie_id').withColumn("index", monotonically_increasing_id())

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [ ]:
pred_df=reduce(lambda x,y:x.join(y,['index'],how='full'),[pl.selectExpr('index',f'prediction{i+1}') for i, pl in enumerate(pred_list)])
pred_df =pred_df.withColumn('predictions',concat_ws(" ",*['prediction%d' %i for i in range(1,len(pred_list)+1)])).select('predictions','index')
outputfile = pred_df.join(movie_list, on=['index'], how='inner')
outputfile.select('movie_id','predictions').toPandas().to_csv('outputpart1.csv',index=False,header=True)

## TF-IDF

In [15]:
from pyspark.ml.feature import HashingTF, IDF

hashingTF = HashingTF(inputCol="plot_output", outputCol="rawFeatures")
train_data_tf = hashingTF.transform(train_data)
# alternatively, CountVectorizer can also be used to get term frequency vectors

idf = IDF(inputCol="rawFeatures", outputCol="features",minDocFreq=100)
idfModel = idf.fit(train_data_tf)
train_data_tf = idfModel.transform(train_data_tf)

train_data_tf.show()


+--------+--------------------+--------------------+--------------------+--------------------+
|movie_id|         plot_output|            map_list|         rawFeatures|            features|
+--------+--------------------+--------------------+--------------------+--------------------+
|18549958|[mary, lonely, po...|[1, 0, 0, 0, 0, 0...|(262144,[14699,16...|(262144,[14699,16...|
| 3150865|[porky, pig, tryi...|[0, 1, 0, 0, 0, 0...|(262144,[9129,120...|(262144,[9129,120...|
| 4345210|[beginning, west,...|[0, 0, 1, 0, 0, 0...|(262144,[3834,420...|(262144,[3834,420...|
|14072881|[seventeen, year,...|[1, 0, 0, 0, 0, 0...|(262144,[1133,183...|(262144,[1133,183...|
| 4711636|[jaded, pilot, na...|[0, 0, 0, 0, 0, 0...|(262144,[571,2089...|(262144,[571,2089...|
|27181651|[hadassah, beauti...|[1, 0, 1, 0, 0, 0...|(262144,[329,1998...|(262144,[329,1998...|
|26551541|[boys, way, home,...|[0, 1, 0, 0, 1, 0...|(262144,[9706,138...|(262144,[9706,138...|
|24070825|[film, takes, pla...|[1, 0, 1, 0, 0, 1..

In [16]:
hashingTF = HashingTF(inputCol="plot_output", outputCol="rawFeatures")
test_data_tf = hashingTF.transform(test_data)
# alternatively, CountVectorizer can also be used to get term frequency vectors

idf = IDF(inputCol="rawFeatures", outputCol="features",minDocFreq=100)
idfModel = idf.fit(test_data_tf)
test_data_tf = idfModel.transform(test_data_tf)

test_data_tf.show()


+--------+--------------------+--------------------+--------------------+
|movie_id|         plot_output|         rawFeatures|            features|
+--------+--------------------+--------------------+--------------------+
| 1335380|[film, based, eve...|(262144,[1728,261...|(262144,[1728,261...|
|29062594|[group, teenagers...|(262144,[6068,191...|(262144,[6068,191...|
| 9252321|[story, zulu, fam...|(262144,[1598,208...|(262144,[1598,208...|
|13455076|[stooges, play, t...|(262144,[3294,618...|(262144,[3294,618...|
|24165951|[soldier, fortune...|(262144,[4098,644...|(262144,[4098,644...|
| 1925869|[set, northwester...|(262144,[535,3294...|(262144,[535,3294...|
|10799612|[like, many, moni...|(262144,[5053,538...|(262144,[5053,538...|
|28238240|[mickey, scorpion...|(262144,[23060,30...|(262144,[23060,30...|
|17124781|[desert, wilderne...|(262144,[5232,733...|(262144,[5232,733...|
|28207941|[bimbo, koko, sig...|(262144,[9726,626...|(262144,[9726,626...|
|19174305|[tahaan, lives, g...|(262144

### Model

In [17]:
dlist = train_data_tf.columns
train_data_tf=train_data_tf.select(dlist+[(col("map_list")[x]).alias("genre"+str(x+1)) for x in range(0, 20)])


In [18]:
# (trainset_tfidf, valset_tfidf) = train_data_tf.randomSplit([0.8, 0.2], seed = 121)

In [19]:
pred_list=[]
for i in range(1,21):
    
    lr = LogisticRegression(featuresCol = 'features', labelCol='genre%s'%i,maxIter=5)
    lrModel = lr.fit(train_data_tf)
    
    predictions_test = lrModel.transform(test_data_tf)
    predictions_test = predictions_test.withColumn("prediction", predictions_test["prediction"].cast(IntegerType()))
    predictions_test = (predictions_test.withColumnRenamed('prediction','prediction%s'%i))
    pred_list.append(predictions_test.select('prediction%s'%i).withColumn("index", monotonically_increasing_id()))
    print(i)
movie_list=predictions_test.select('movie_id').withColumn("index", monotonically_increasing_id())

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [ ]:
pred_df=reduce(lambda x,y:x.join(y,['index'],how='full'),[pl.selectExpr('index',f'prediction{i+1}') for i, pl in enumerate(pred_list)])
pred_df =pred_df.withColumn('predictions',concat_ws(" ",*['prediction%d' %i for i in range(1,len(pred_list)+1)])).select('predictions','index')
outputfile = pred_df.join(movie_list, on=['index'], how='inner')
outputfile.select('movie_id','predictions').toPandas().to_csv('outputpart2.csv',index=False,header=True)

## Word2Vec

In [20]:
from pyspark.ml.feature import Word2Vec

word2Vec = Word2Vec(vectorSize=200, minCount=5, inputCol="plot_output", outputCol="features")
model = word2Vec.fit(train_data)

train_data_wv = model.transform(train_data)
train_data_wv.show()


+--------+--------------------+--------------------+--------------------+
|movie_id|         plot_output|            map_list|            features|
+--------+--------------------+--------------------+--------------------+
|18549958|[mary, lonely, po...|[1, 0, 0, 0, 0, 0...|[0.08346132726364...|
| 3150865|[porky, pig, tryi...|[0, 1, 0, 0, 0, 0...|[-0.0229134946882...|
| 4345210|[beginning, west,...|[0, 0, 1, 0, 0, 0...|[0.12968615381396...|
|14072881|[seventeen, year,...|[1, 0, 0, 0, 0, 0...|[0.03247320896284...|
| 4711636|[jaded, pilot, na...|[0, 0, 0, 0, 0, 0...|[0.03273211191200...|
|27181651|[hadassah, beauti...|[1, 0, 1, 0, 0, 0...|[0.06491560155602...|
|26551541|[boys, way, home,...|[0, 1, 0, 0, 1, 0...|[-0.0139750445984...|
|24070825|[film, takes, pla...|[1, 0, 1, 0, 0, 1...|[0.09414260529068...|
|30897463|[film, tells, sto...|[0, 0, 1, 0, 0, 0...|[0.09715022520641...|
| 5349196|[arul, vikram, wo...|[1, 0, 0, 0, 0, 1...|[0.03366250111255...|
|32751781|[australian, coun...|[1, 0, 

In [21]:
word2Vec = Word2Vec(vectorSize=200, minCount=5, inputCol="plot_output", outputCol="features")
model_test = word2Vec.fit(test_data)

test_data_wv = model.transform(test_data)
test_data_wv.show()


+--------+--------------------+--------------------+
|movie_id|         plot_output|            features|
+--------+--------------------+--------------------+
| 1335380|[film, based, eve...|[0.05598512597684...|
|29062594|[group, teenagers...|[0.14241938944905...|
| 9252321|[story, zulu, fam...|[0.07958495482291...|
|13455076|[stooges, play, t...|[0.07475852966925...|
|24165951|[soldier, fortune...|[-0.0557895397651...|
| 1925869|[set, northwester...|[0.02304582834951...|
|10799612|[like, many, moni...|[0.08219743698370...|
|28238240|[mickey, scorpion...|[0.01957262284122...|
|17124781|[desert, wilderne...|[0.00473414125503...|
|28207941|[bimbo, koko, sig...|[0.05876081616484...|
|19174305|[tahaan, lives, g...|[0.04216506349735...|
|18392317|[betty, startled,...|[0.04898787961876...|
|34420857|[nirmal, karthik,...|[0.03164040526873...|
| 4039635|[group, journalis...|[0.05018935955401...|
| 8034072|[vaibhavari, saha...|[0.06937972646987...|
| 4016437|[1947, movie, nar...|[0.081862672087

## Model

In [22]:
dlist = train_data_wv.columns
train_data_wv=train_data_wv.select(dlist+[(col("map_list")[x]).alias("genre"+str(x+1)) for x in range(0, 20)])


In [23]:
# (trainset_wv, valset_wv) = train_data_wv.randomSplit([0.8, 0.2], seed = 121)

In [24]:
pred_list=[]
for i in range(1,21):
    
    lr = LogisticRegression(featuresCol = 'features', labelCol='genre%s'%i,maxIter=5)
    lrModel = lr.fit(trainset_wv)
    
    predictions_test = lrModel.transform(test_data_wv)
    predictions_test = predictions_test.withColumn("prediction", predictions_test["prediction"].cast(IntegerType()))
    predictions_test = (predictions_test.withColumnRenamed('prediction','prediction%s'%i))
    pred_list.append(predictions_test.select('prediction%s'%i).withColumn("index", monotonically_increasing_id()))
    print(i)
movie_list=predictions_test.select('movie_id').withColumn("index", monotonically_increasing_id())

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [ ]:
pred_df=reduce(lambda x,y:x.join(y,['index'],how='full'),[pl.selectExpr('index',f'prediction{i+1}') for i, pl in enumerate(pred_list)])
pred_df =pred_df.withColumn('predictions',concat_ws(" ",*['prediction%d' %i for i in range(1,len(pred_list)+1)])).select('predictions','index')
outputfile = pred_df.join(movie_list, on=['index'], how='inner')
outputfile.select('movie_id','predictions').toPandas().to_csv('outputpart3.csv',index=False,header=True)